# User-to-user Collaborative Filtering

Create a basic user-to-user K-nearest neighbors (u2uknn) collaboritive filtering solution for recommended tracks.
This is built from the Verstrepen2017 matrix notation of a score matrix factored by user-to-user similarity and the ratings matrix.

    Score = Similarity x Ratings 
    
The similarity and ratings matrix are built from an append of the challenge set to the training set.
This is required by KNN because the similarity computation is of each individual challenge playlist against all the training set.
In this formulation we technically also include the other training set similarities in the final score.
This simplifies the computation and dimension managment and avoids having to loop on individual challenge playlist and training set pairings.
We assume that the impact on final scores is minimal.

The cold start task is address with a simple global popularity ranking of tracks.
We compute the global popularity of tracks and recommend the first 500 in ranked order.
This set is also included as backfill to each playlist recommendation accross all subtasks to ensure the minimum 500 tracks are available to each challenge solution.

In [ ]:
import sys
import json
import re
import collections
import os
import datetime
import pandas as pd
import numpy as np
from datetime import date, datetime
import utils
from utils import tic, toc
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split
import torch
from torch import nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

## Set parameters for Run

These parameters control the selection of dataset, challenge set, and solution output.

Must select dataset and challege_name.  The rest are derived parameters and can be left alone.

In [ ]:
# path to the directory with different collections of training and test sets
datadir="data/"  # path to base data set
resultsdir=""

# select the data for training  
#dataset="mpd/data"     # the original full mpd training set, requires quick=True and max_files set
#dataset="mpd-1st-21k"  # first 21 files of mpd, equiv to quick=True and max_files 20
#dataset="mpd-2nd-21k"  # second 21 files of mpd, avoids using data that built mympd challenge set
dataset="mympd-full-20k"

# select data for testing, this is the challenge_set.json file for specific challenge data
#challenge_name="mpd"  # original mpd challenge set, use with aicrowd
#challenge_name="mympd"  # my custom challenge set for task analysis
challenge_name="mympd-full"  # my custom challenge sampled from full training set for task analysis

# optionally tag the run
# this can be used to group the results file with other run artifacts, eg. job info
tagname=""


# provide method selector to do u2u or i2i collaborative filtering
method="nn-ae"

# train against tracks with minimum support, all by default
minsupport = 0

#### Set the model parameters and then train

We can control the batch size with the data loaders.  An can set the other parameters based on exising models of interest. (e.g. hello world)

    [DAE]
    epochs = 20
    batch = 250
    lr = 0.005
    reg_lambda = 0.0
    hidden = 256

In [ ]:
#
# Parameters from hello world! DAE
# 
epochs = 20
batch_size = 250
learning_rate = 0.005

The papermill parameter cell needs to set variables exclusively before they can be used.  The CLI parameters are injected in a cell after the parameters cell.  The parameters can't be used until after the injected cell, otherwise they just get the static default values.

In [ ]:
# derived parameters, no need to change.

if (len(tagname)>0): tagname="_"+tagname

#trainset="data/mpd/data"  # relative path to training set in slice json file format
trainset=datadir+dataset+"/data"  # relative path to training set in slice json file format

#testset="data/challenge_set.json" # relative path to test set in challenge_set.json format
#testset=datadir+challenge_name+"-challenge-set/challenge_set.json" # relative path to test set in challenge_set.json format
testset=datadir+challenge_name+"/challenge_set.json" # relative path to test set in challenge_set.json format

datestr=datetime.now().strftime('%Y-%m-%d-%H:%M:%S')

challenge_header = "team_info,jprorama,jprorama@gmail.com\n"
challenge_solution = resultsdir+"method-"+method+"_"+challenge_name+"_"+dataset+"_"+datestr+tagname+".csv"

# checkpoint file for model
modelchk = resultsdir+"/cache/"+"method-"+method+"_"+challenge_name+"_"+dataset+".pt"

In [ ]:
# generate more verbose output for some functions
debug = True

# parameters for mpd load, superceeded by mpd-1st-21k
quick = False
max_files_for_quick_processing = 20

# random state
seed = 1

## Load the mpd training data

Create data frame for playlists and tracks to make it simple to work with the training data.

In [ ]:
%%time
playlists, tracks = utils.process_mpd(trainset, quick, maxfiles=max_files_for_quick_processing)

Set a new index for playlists so each row has unique id using pid. After reading the slice files the index values repeat for each slice.

Preference is to not use the pid since that drops this data column.
Instead create a new column of integers for each row and then set that as the index.

In [ ]:
playlists

In [ ]:
trainset_size=playlists.shape[0]

In [ ]:
pl = playlists.copy()

In [ ]:
pl = playlists[["pid","tracks"]].explode("tracks")

In [ ]:
pl["track_uri"] = [d.get("track_uri") for d in pl.tracks]

In [ ]:
pl["artist_name"] = [d.get("artist_name") for d in pl.tracks]

The expanded one-row-per-track representation shows we have 1.4million songs (rows). The row index has 21k entries which matches the 21k playlists in the training set.

In [ ]:
pl

### Check memory usage

In [ ]:
tracks = pl[["track_uri"]]

In [ ]:
tracks.memory_usage(deep=True)

In [ ]:
tracks.memory_usage()

In [ ]:
pd.__version__

In [ ]:
tracks.info()

From example in [pandas sparse data types page](https://pandas.pydata.org/pandas-docs/stable/user_guide/sparse.html) use memory_usage().sum().  Not clear why we divide by 1000.  Would think that makes it kilobytes.

In [ ]:
'dense : {:0.2f} kbytes'.format(tracks.memory_usage().sum() / 1e3)

## One Hot encode playlists

Attempting to use get_dummies() works in the dense space an tries to build a dataframe of 100k by 1.4Million songs.  Not sure why so many rows but it's still to big for ram at 300+G

trackhots = pd.get_dummies(tracks, dtype=bool)

sklearn has a onehot encoder that is a preprocessor to many of its routines.  See if we can fit the tracks to this representaiton.

https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MultiLabelBinarizer


pl[["pid", "track_uri"]].info()

trackhots = OneHotEncoder()

trackhots.fit(pl[["pid", "track_uri"]])

trackhots.categories_

trackhots.get_feature_names()

Transform the original data into a matrix representation.

Here again is the 1.4x290k represenation.  The 1.4k is the songs, so rows in the original matrix but not clear where the 290k comes from.  Would expect 21k for the playlists.

th = trackhots.transform(pl[["pid","track_uri"]])

th

pl["pid"].max()

playlists["num_tracks"].sum()

Hmm, there are some problems in the transformation.  The 1.4mil comes from the total number of tracks in training.  The total unique is much smaller.

pl["track_uri"].drop_duplicates().count()

I'd expect an transformed data set to be 21k by 269k.

Ah, the onehot encoder wants a feature set of each record with its distinct features.
https://scikit-learn.org/stable/modules/preprocessing.html#preprocessing-categorical-features

in this case it's rows of track_uri.
so each row with mapp to the idx value and will just have tracks.

playlists.head(1)

In [ ]:
import ast

pl[["track_uri"]]

Try converting each tracks string to a data type 

https://www.geeksforgeeks.org/python-convert-string-dictionary-to-dictionary/

playlists[["tracks"]].tracks

We need a list of lists. This is pretty easy to construct with a list comprehension to wrap the lists into a list.

pltracks = [d for d in playlists[["tracks"]].tracks.apply((lambda s: [d["track_uri"] for d in s]))]

len(pltracks)

type(pltracks)

what we are really trying to do is train the encoding and then transform each row.

this is more like having a vocabulary and different sentances.
I need to map each sentance to it's onehot encoding of the vocabulary.

this example shows moving from an integerencoding to a one hot encoding
https://machinelearningmastery.com/how-to-one-hot-encode-sequence-data-in-python/

reading the docs leads to multi label binarizer which appears to be closer to what i want.
https://scikit-learn.org/stable/modules/preprocessing_targets.html#multilabelbinarizer

In [ ]:
mlb = MultiLabelBinarizer(sparse_output=True)

pltracks = mlb.fit_transform(pltracks)

We finally have a list of 21k playlists encoded with the 269k unique tracks.`

pltracks

## Get Cosine similarity

https://stackoverflow.com/a/27046041/8928529

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity


sim = cosine_similarity(pltracks)

sim.shape

We want to do a matrix multiply for user-user similarity: score = sim * ratings

https://stackoverflow.com/a/16754459/8928529

In [ ]:
from scipy import sparse

cast the similarity martrix into a compressed sparse row format so matrix multiplication doesn't explode the ram.

sim = sparse.csr_matrix(sim)

sim

score = sparse.csr_matrix.dot(sim, pltracks)

The result is a score matrix in the original dimentions that is 17% sparse.  With 973mil out of 5billion possible

score

score.shape

## Append Challenge Set to Training Set

Read the data from the challenge set file and append the tracks to training tracks in prep for similarity comparison. Omit first 1000 tracks since this is the title only subtask. Their similarity is implicitly zero on all tracks

In [ ]:
# load data using Python JSON module
with open(testset,'r') as f:
    data = json.loads(f.read())

In [ ]:
# Flatten data
challenge_playlists = pd.json_normalize(data, record_path=['playlists'])

In [ ]:
[challenge_playlists["tracks"]]

In [ ]:
chtracks = [d for d in challenge_playlists[["tracks"]].tracks.apply((lambda s: [d["track_uri"] for d in s]))]

In [ ]:
chtracks[1002]

In [ ]:
pltracks = [d for d in playlists[["tracks"]].tracks.apply((lambda s: [d["track_uri"] for d in s]))]

In [ ]:
pltracks[0]

In [ ]:
chtracks[1000]

In [ ]:
alltracks = list()

In [ ]:
alltracks = pltracks + chtracks[1000:]

In [ ]:
len(alltracks)

### Memory size of matrices

https://pretagteam.com/question/determining-the-byte-size-of-a-scipysparse-matrix

In [ ]:
def sparse_bytes(a):
    return a.data.nbytes + a.indptr.nbytes + a.indices.nbytes

In [ ]:
def parts_bytes(a):
    return [a.data.nbytes, a.indptr.nbytes, a.indices.nbytes]

In [ ]:
def parts_types(a):
    return [a.data.dtype, a.indptr.dtype, a.indices.dtype]

In [ ]:
def in_megs(n):
    
    MB=n/1024/1024
    return "{:6.2f} MB".format(MB)

In [ ]:
def size_report(a, name="matrix"):
    print("shape of {}: {}".format(name, a.shape))
    print("nnz of {}: {}".format(name, a.nnz))
    print("sparsity of {}: {:3.4f} %".format(name, 100*(1-a.nnz/(a.shape[0]*a.shape[1]))))
    print("size of {}: {}".format(name, in_megs(sparse_bytes(a))))
    print("size of {} parts: data: {}, indptr: {}, indices: {}".format(name, *map(in_megs, parts_bytes(a))))
    print("type of {} parts: data: {}, indptr: {}, indices: {}".format(name, *parts_types(a)))

## Create Sparse Binary User-Item Matrix

In [ ]:
allmpd = mlb.fit_transform(alltracks)

In [ ]:
allmpd

In [ ]:
size_report(allmpd, "all_ratings")

## Filter tracks without minimum support

By default we use all tracks to build our models.  This doesn't necessarily make sense for tracks that appear in only one playlist since these are not good candidates for recommendation.  How do you recommend something that only one person ever liked. By definition these tracks would not show up in another playlist.

These tracks are also expensive to maintain in the model because they account for almost half of the tracks in a sample.  Including them, therefore, almost doubles the lenth of the vectors we must maintain in in our models.  By elliminiting these tracks we significantly reduce the memory footprint of the weight matrices in our models.

It's easiest to discover these tracks by summing our initial non-filtered collection of tracks and using that as input to a second pass at a model that has a restricted set of classes given to the multi-label binarizer.

In [ ]:
if (minsupport > 0):
    
    # create boolean filter for 
    minsupport_filter = allmpd.sum(axis=0)>minsupport
    minsupport_filter = np.squeeze(np.asarray(minsupport_filter))

    print(f"original track count: {len(mlb.classes_)}")
    print(f"filtered track count: {len(mlb.classes_[minsupport_filter])}")

    mlb_filter = MultiLabelBinarizer(classes=mlb.classes_[minsupport_filter], sparse_output=True)

    allmpd = mlb_filter.fit_transform(alltracks)

In [ ]:
allmpd

In [ ]:
size_report(allmpd, "all_ratings")

In [ ]:
size_report(allmpd[0:trainset_size], "train_ratings")

In [ ]:
size_report(allmpd[trainset_size:], "test_ratings")

### Create a train-test split

The AENN needs a test set to asses performance during training.

We follow the lead of hello-world! and use a 3k test set

In [ ]:
allmpd_train, allmpd_test = train_test_split(allmpd[:trainset_size], test_size=3000, random_state=seed)

In [ ]:
allmpd_train.shape

In [ ]:
allmpd_test.shape

### Create a dataset and loader for torch

The dataset and loader are based on this [example in the tutorial](https://pytorch.org/tutorials/beginner/basics/data_tutorial.html).  The examples describe loading data from files within the dataset \__getitem__ but we already have all that parsing complete.  We can simply load the result parsed data into the class and convert it to a sparse tensor.  This makes it easy for the dataset and dataloader to iterate through the object for training.

In [ ]:
class allmpdDataset(Dataset):
    def __init__(self, mpd):
        '''
        convert the input matrix from scipy csr to torch sparse via COO intermediary format
        https://discuss.pytorch.org/t/creating-a-sparse-tensor-from-csr-matrix/13658/5
        
        Note the explicit dimension setting arg via the coo.shape. This ensures
        the matrix doesn't lose dimensions due to the max column values being different.
        This occurs if the max column id happens to be less that the global max, ie. 
        a missing max item id in one of the conversions.
        '''
        coo = mpd.tocoo()
        values = coo.data
        indices = np.vstack((coo.row, coo.col))

        i = torch.LongTensor(indices)
        v = torch.FloatTensor(values)
        shape = coo.shape

        self.mpd = torch.sparse.FloatTensor(i, v, torch.Size(shape))

    def __len__(self):
        return self.mpd.shape[0]

    def __getitem__(self, idx):        
        return self.mpd[idx]

Instantiate the class with the parsed data

In [ ]:
mpd_train = allmpdDataset(allmpd_train)

In [ ]:
mpd_test = allmpdDataset(allmpd_test)

Create the dataloader to iterrate though the mpd dataset. Start with a small batch to observe operation. Start with non-random iteration for repeatability.

In [ ]:
dl_train = DataLoader(mpd_train, batch_size=64, shuffle=False)

In [ ]:
dl_test = DataLoader(mpd_test, batch_size=64, shuffle=False, drop_last=True)

Show the first batch of data as confirmation of working iterable.

In [ ]:
next(iter(dl_train))

In [ ]:
next(iter(dl_test))

## Build a Simple AutoEncoder

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

### Build the AutoEncoder class

Build the network using the layers proved out above.  The structure is driven by a desire to build a basic encoder-decoder for playlists.  The network is designed to use just the user-item matrix as represented by playlist-song data.  No additional data is used in order to facilitate direct comparison to mf-als and the knn methods.

In [ ]:
class AE(nn.Module):
    def __init__(self, in_features, embedding=256):
        super(AE, self).__init__()
        
        self.aestack = nn.Sequential(
            nn.Linear(in_features=in_features, out_features=embedding),
            nn.Sigmoid(),
            nn.Linear(in_features=embedding, out_features=in_features),
            nn.Sigmoid()
        )
        
    def forward(self, x):
        return self.aestack(x)
        
                  

The model and data need to be on the same device (gpu or cpu) in order to run through the network via the forward pass.

In [ ]:
torch.cuda.memory_allocated()

### Inspect the model model

We can access all the parameters of the model at each layer.
https://discuss.pytorch.org/t/access-all-weights-of-a-model/77672/2

for name, param in model.named_parameters():
    print(f"name: {name}")

The size the model weights and biases take up on the gpu grows as the size of the input grows.
The input is sparse but the wieght matrices become dense.  Additionally the output of the batch is dense.

In [ ]:
def model_size_report(model):
    w1bytes = model.aestack[0].weight.size()[0] * model.aestack[0].weight.size()[1] * 4
    b1bytes = model.aestack[0].bias.size()[0] * 4
    w2bytes = model.aestack[2].weight.size()[0] * model.aestack[2].weight.size()[1] * 4
    b2bytes = model.aestack[2].bias.size()[0] * 4
    outbytes = batch_size * model.aestack[2].weight.size()[0] * 4

    print(f"w1bytes: {w1bytes}")
    print(f"b1bytes: {b1bytes}")
    print(f"w2bytes: {w2bytes}")
    print(f"b2bytes: {b2bytes}")
    print(f"modelbytes: {w1bytes+b1bytes+w2bytes+b2bytes}")
    print(f"outbytes: {outbytes}")
    print(f"batchsize: {batch_size}")
    print(f"totalgpubytes: {w1bytes+b1bytes+w2bytes+b2bytes+outbytes}")

model_size_report(model)

torch.cuda.memory_allocated()

## Build the Training and Test Loops

In [ ]:
debug = False
if (device == "cuda"):
    memuse = True
else:
    memuse = False

In [ ]:
def train_loop(dataloader, model, loss_fn, optimizer, epoch=1):
    global memuse
    global modelchk

    start_event = torch.cuda.Event(enable_timing=True)
    end_event = torch.cuda.Event(enable_timing=True)

    size = len(dataloader.dataset)
    for batch, X in enumerate(dataloader):
                
        if (batch > 3): memuse = False

        if (memuse):
            start_event.record()

        # Compute prediction and loss
        if (memuse): print(f"gpu mem pre-batch: {torch.cuda.memory_allocated()}")
        X = X.to(device)
        if (memuse): print(f"gpu mem post-batch: {torch.cuda.memory_allocated()}")
        pred = model(X)
        if (memuse): print(f"gpu mem post-pred: {torch.cuda.memory_allocated()}")

        X = X.to_dense()
        if (memuse): print(f"gpu mem post-X dense: {torch.cuda.memory_allocated()}")
        pred = pred.to(device)
        if (memuse): print(f"gpu mem post-pred to device: {torch.cuda.memory_allocated()}")

        if (debug):
            print(f"X type: {type(X)}")
            print(f"pred type: {type(pred)}")
            print(f"X is sparse: {X.is_sparse}")
            print(f"pred is sparse: {pred.is_sparse}")
            # size of tensor
            # https://discuss.pytorch.org/t/how-to-know-the-memory-allocated-for-a-tensor-on-gpu/28537/2
            print(f"X size: {X.element_size() * X.nelement()}")
            print(f"pred size: {pred.element_size() * pred.nelement()}")

       
        # The autoencoder trains against the input samples
        if (memuse): print(f"gpu mem pre-loss: {torch.cuda.memory_allocated()}")
        loss = loss_fn(pred, X)
        if (memuse): print(f"gpu mem post-loss: {torch.cuda.memory_allocated()}")


        # Backpropagation
        optimizer.zero_grad()
        if (memuse): print(f"gpu mem pre-backward: {torch.cuda.memory_allocated()}")
        loss.backward()
        if (memuse): print(f"gpu mem post-backward: {torch.cuda.memory_allocated()}")
        optimizer.step()
        if (memuse): print(f"gpu mem post-step: {torch.cuda.memory_allocated()}")

        if (memuse):
            end_event.record()
            torch.cuda.synchronize()  # Wait for the events to be recorded!
            elapsed_time_ms = start_event.elapsed_time(end_event)
            print(f"elapsed time ms: {elapsed_time_ms}")

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
            
    # checkpoint at end of epoch
    torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss,
            }, modelchk)

In [ ]:
def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X in dataloader:
            if (debug):
                print(f"X test size: {X.shape}")
            X = X.to(device)
            if (debug):
                print(f"X test size: {X.shape}")
            pred = model(X)
            pred = pred.to(device)
            X = X.to_dense()

            test_loss += loss_fn(pred, X).item()
            correct += (pred.argmax(0) == X).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

## Run the model from a fresh start with multiple epochs of training

Now we can observe the impact of the learning rate, loss function and optimizer selected above in a full training run.

First we reset the model to start the training process from the initial point.

In [ ]:
torch.cuda.memory_allocated()

In [ ]:
model = AE(allmpd_train.shape[1]).to(device)

In [ ]:
torch.cuda.memory_allocated()

In [ ]:
dl_train = DataLoader(mpd_train, batch_size=batch_size, shuffle=False)
dl_test = DataLoader(mpd_test, batch_size=batch_size, shuffle=False)

In [ ]:
loss_fn = nn.BCELoss()

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
model_size_report(model)

In [ ]:
torch.cuda.memory_allocated()

In [ ]:
torch.cuda.memory_allocated()/1024/1024/1024

In [ ]:
torch.cuda.memory_stats()

In [ ]:
cur_epoch = 0
if (os.path.exists(modelchk)):
    checkpoint = torch.load(modelchk)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    cur_epoch = checkpoint['epoch']
    loss = checkpoint['loss']
    
    print(f"Epoch {cur_epoch}\n-------------------------------")
    print(f"loss: {loss:>7f}")
    
    if (cur_epoch < epochs):
        # continue training
        model.train()
    else:
        # ready for inference
        model.eval()

In [ ]:
cur_epoch

In [ ]:
%%time

for t in range(cur_epoch, epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    if (device == "cuda"): memuse = True
    train_loop(dl_train, model, loss_fn, optimizer, epoch=t+1)
    test_loop(dl_test, model, loss_fn)
print("Done!")

### Assess the quality of the learning

Sum the weights and biases to confirm that they changed since before training.

Also compare the number of unique weight values to the total possible values for the trained model. 

In [ ]:
w1sum_post = model.aestack[0].weight.sum()
b1sum_post = model.aestack[0].bias.sum()
w2sum_post = model.aestack[2].weight.sum()
b2sum_post = model.aestack[2].bias.sum()

print(f"w1 sum: {w1sum_post}")
print(f"b1 sum: {b1sum_post}")
print(f"w2 sum: {w2sum_post}")
print(f"b2 sum: {b2sum_post}")

In [ ]:
model.aestack[0].weight.size()[0] * model.aestack[0].weight.size()[1]

In [ ]:
len(model.aestack[0].weight.unique())

In [ ]:
len(model.aestack[2].weight.unique())

## Compute Recommendations from model

### Create a dataset and data loader for the challenge tracks.

The create a function to loop through the challenge dataset and gather the recomendations.

The recommendations can then be used directly to produce recommendations via torch.argsort() which produces the index of entries in sorted order.

In [ ]:
allmpd[trainset_size:][0].indices

In [ ]:
mpd_chall = allmpdDataset(allmpd[trainset_size:])

In [ ]:
dl_chall = DataLoader(mpd_chall, batch_size=batch_size, shuffle=False)

### Visually inspect samples at 0 and 1000 to ensure that challenge set mapping is accurate

We want to make sure that the wrapping of the allmpd challenge track remains consistent in all its different representations: scipy sparse, sparse tensor, and data loader wrapper.

Inspect index 0 and index 1000 to confirm they contain the same track id values as their index.

In [ ]:
allmpd[trainset_size:][0].indices

In [ ]:
mpd_chall[0]

In [ ]:
next(iter(dl_chall))[0]

In [ ]:
allmpd[trainset_size:][1000].indices

In [ ]:
mpd_chall[1000]

In [ ]:
# compute the iterations and offset to get to 
# challenge playlist 1000 with the dataloader

batch = 1000 // dl_chall.batch_size
offset = 1000 - (dl_chall.batch_size * batch)

for i, X in enumerate(dl_chall):
    if (i==batch):
        print(X[offset])
        break

### Compute the Challenge recommendations

In [ ]:
def rec_loop(dataloader, model):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    
    # build a list of recs from the batch loop
    # https://pytorch.org/docs/stable/generated/torch.cat.html#torch-cat
    recs = []
    
    with torch.no_grad():
        for i, X in enumerate(dataloader):
            if (debug):
                print(f"X test size: {X.shape}")
            X = X.to(device)
            if (debug):
                print(f"X test size: {X.shape}")
            recs.append(model(X).to("cpu"))
            
    return torch.cat(recs)


In [ ]:
recs = rec_loop(dl_chall, model)

### Inspect the challenenge recommendations

The dimensions are correct but all the track recommendations are identical

In [ ]:
recs.size()

In [ ]:
recs[0].argsort()

In [ ]:
recs[1].argsort()

In [ ]:
recs[1000].argsort()

In [ ]:
recs[1001].argsort()

In [ ]:
model.aestack

In [ ]:
model.type

So the sort() an argsort() return their results in ascending order by score. That means it's the lowest to highest score, or worst to best recommendation.

In [ ]:
recs[0].sort()

In [ ]:
recs[0].argsort()

Add the descending arg to reverse the sort order and return the preferred result.

In [ ]:
recs[0].sort(descending=True)

In [ ]:
recs[0].argsort(descending=True)

## Compute Top-N Tracks

Use the global top-N tracks to backfill missing data for playlist recommendation.
The top-n global also forms a baseline recommender to assess overall performance.

We focus on the global track stats for the training dataset only. It makes sense because this is the known data but also because the final submission requires removal of challenge seed tracks so any track information learned from the challenge set can't be reused in the same list. Worth questioning some since it can't be reused in the same seed but could be in other playlists...

However, do we need the full corpus so that we can match up the vocabulary terms to the tracks?  No because we can just use the Vecotrizors vocab.

Approach is to treat the training playlists as corpus of text documents.  Count the occurance of terms with the count vectorizer.  Then sum the term counts into a single array.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from scipy.sparse import csr_matrix

Consruct corpus from playlists joined a list of strings.

In [ ]:
[ " ".join(doc) for doc in alltracks[0:2]]

We use a custom tokenizer to avoid splitting on the punctuation characters in the "spotify:track:xxx' pattern. https://stackoverflow.com/a/37884104/8928529

In [ ]:
vectorizer = CountVectorizer(tokenizer=lambda x: x.split(' '))

In [ ]:
trackcounts = vectorizer.fit_transform([" ".join(doc) for doc in alltracks[0:20000]])

In [ ]:
trackcounts

In [ ]:
pd.DataFrame.sparse.from_spmatrix(trackcounts)

In [ ]:
trackfreq = trackcounts.T.sum(axis=1)

In [ ]:
len(trackfreq)

In [ ]:
trackfreq.shape

In [ ]:
pd.DataFrame(trackfreq)

In [ ]:
trackfreq

In [ ]:
dtype = [("trackidx", int), ("count", int)]
new = np.empty(len(trackfreq), dtype)

In [ ]:
new

In [ ]:
np.array([*range(len(trackfreq))]).astype(int)

In [ ]:
new['trackidx'] = np.array([*range(len(trackfreq))])

In [ ]:
np.array(trackfreq).reshape(len(trackfreq)).shape

In [ ]:
new['count'] = np.array(trackfreq).reshape(len(trackfreq))
#print new

In [ ]:
new

In [ ]:
pd.DataFrame(new)

In [ ]:
tuples = zip(new["trackidx"], new["count"])

In [ ]:
topn = sorted(tuples, key=lambda x: (x[1]), reverse=True)

In [ ]:
topn

In [ ]:
vectorizer.vocabulary_

Invert the vocabulary dictionary so we can map it to the tracks. https://stackoverflow.com/a/483833/8928529

In [ ]:
inv_map = {v: k for k, v in vectorizer.vocabulary_.items()}

In [ ]:
inv_map

Generate a list of the topn track identifiers

In [ ]:
toptracks=[inv_map[i] for i, cnt in iter(topn)]

In [ ]:
toptracks = toptracks[0:1000]

The top tracks can now be appended to each list an ensure we have the minimum recommenadable track set.

## Explore filtering challenge tracks from recommendation list

Remove the challenge tracks from the recommended set.
Use a simple loop for now to keep the code simple.
Also allows us to inspect where the original songs are in the recommendation set.
For the current playlist, app positions for the first 5 songs are above the 500 song rec limit.
This suggests we will see a fairly poor rprec and ndcg performance for pure user-user knn.
Makes sense, since this is really just a most popular songs amoung similar users strategey.
A user focused popularity ranking rather than a global popularity ranking.
Suggests the need for the boosting strategies we see in the actually top performers.

len(rectracks)

Clean up the entire recommendation set. This is lists 21000-30000 in the currrent method. No index math is needed if we shift to putting the challenge tracks at the start.

Trim the recommenation set out of the score results

Even with the del earlier the RES memory remains  at 28g which helps explain why the next step kills the kernel

    98895 jpr       20   0   29.9g  28.7g  29564 S   0.0 15.3   4:06.54 python

Even with explicit garbage collection this doesn't free up the ram.

https://stackoverflow.com/questions/1316767/how-can-i-explicitly-free-memory-in-python

Advice is to use a subprocess.

In [ ]:
import gc

gc.collect()

Maybe it means I need to save out the results and reload them either in a new notebook or after the kernel barfs.

In [ ]:
import pickle

pickle.dump(score, open( "save_score.p", "wb" ) )

score=pickle.load(open( "save_score.p", "rb" ))

Even pickle kills the kernel. 
Maybe best to just add some ram.

Remove the challenge tracks from the recommendation list.  The recommendation list is augmented with the topn (n=1000) most popular tracks to backfill recommendations that don't have enough tracks to fill the 500 count requirement.

The algorithm slows noticably as the number of challenge tracks increases and the recommendation list has to be search repeatedly.

The 9000 scored playlists start for playlist 1000.
The challenge playlist starts with the first playlist.
Need to offset the challenge playlist index to match the score structure and recommended tracks.

In [ ]:
%%time 

import time

reclist = list()
indexdist = list() #pd.DataFrame(columns=["index"])
misses = 0
tooshort = 0
trace = False
timeit=False

start_time = time.time()
for idx in range(9000):
    # get the candidate track class id sorted by recommendation score
    if (timeit): tic()
    cantracks = recs[idx].argsort(descending=True).tolist()[0:750]
    if (timeit): toc("cantracks")
    
    # convert class id to spotify track name
    if (timeit): tic()
    rectracks=[mlb.classes_[i] for i in cantracks]
    if (timeit): toc("rectracks")
    
    if (timeit): tic()
    # ensure mininum length recommendation meets 500 tracks requirement
    rectracks=rectracks + toptracks
    # truncate rectracks to top1000 to limit search effort for challenge tracks
    rectracks=rectracks[:1000]
    if (timeit): toc("rectracks_1k")
    if (trace): print("idx {}:".format(idx))
    # remove challenge tracks from recommendation list
    # note: the challenge tracks start at index 1000 to allign tasks with recommendations
    
    # convert to dataframe to benefit from vector operations
    # impoves about 2x over speed of manual loops.
    if (timeit): tic()
    recdf = pd.DataFrame(rectracks, columns=["track"])
    if (timeit): toc("recdf")
    if (timeit): tic()
    filterlist = recdf[recdf.track.isin(chtracks[idx+1000])].index
    if (timeit): toc("filterlist")
    # need to record index collection for distribution analysis but this isn't the correct approach
    #indexdist.append(filterlist)
    if (timeit): tic()
    recdf.drop(filterlist, inplace=True )
    if (timeit): toc("recdf.drop")
    if (timeit): tic()
    rectracks = recdf["track"].tolist()
    if (timeit): toc("recdf.tolist")

    #for challenge_track in chtracks[idx+1000]:
    #    if (trace): print("look for track: {}".format(challenge_track))
    #    while challenge_track in rectracks:
    #        try:
    #            indexdist.append(rectracks.index(challenge_track))
    #            if (trace): print("remove track pos: {}".format(rectracks.index(challenge_track)))
    #            rectracks.remove(challenge_track)
    #        except (ValueError, AttributeError):
    #            if (trace): print("didn't find in rectracks: {}".format(challenge_track))
    #            misses += 1

    
    #if reclist < 500:
    #    tooshort += 1
    
    # truncate recommendation list to the 500 length required
    if (timeit): tic()
    reclist.append(rectracks[0:500])
    if (timeit): toc("reclist.append")
    
    # progress bar
    if (idx % 1000) == 0:
        print("--- %s seconds ---" % (time.time() - start_time))
        print("challenge tracks progress: {}".format(idx))
        start_time = time.time()


The index distribution is a simple peek into the performance of the KNN algorithm.  It shows the average index value of the seeded challenge tracks found in the recommendations.  Given that this number is dominated by values above the general playlist lengths and even above 500 it's clear that the similarity measure alone is not an effective way of organizing the recommendation results.

Correction: the challenge set index was misalligned with the recommendation set index.  After updating the index to start after the title only task (+1000) the collection of matches shifted to where 70% of tracks were found in the first 500 recommendations.

indexdist = pd.DataFrame(indexdist)

The distribution of removals shows that the vast majority are well above the 500 reclist limit

indexdist.describe()

In [ ]:
len(reclist)

In [ ]:
misses

In [ ]:
len(reclist[8995])

In [ ]:
submission = pd.DataFrame(reclist)

In [ ]:
submission

# add the top popular to flesh out recommendation

In [ ]:
toptracks[0:5]

In [ ]:
coldstart = pd.DataFrame(columns=[*range(500)])


In [ ]:
coldstart

In [ ]:
coldstart = coldstart.append([toptracks[0:500]])

In [ ]:
coldstart

In [ ]:
%%time

# create data frame of 1000 copies of top popular to substitute for title only cold start recommendation

coldstart = pd.DataFrame(columns=[*range(500)])

for i in range(1000):
    coldstart = coldstart.append([toptracks[0:500]])

In [ ]:
coldstart.shape

In [ ]:
solution = coldstart.append(submission)

In [ ]:
solution.shape

Reset the index so all rows have a distinct index value.  This is necessary so the pid insert below correctly adds each pid to the title only task.  Otherwise the repeated rows of the task are seen as a single distinct row and all get the same pid.

In [ ]:
solution = solution.reset_index(drop=True)

## Add playlist id into the submission

In [ ]:
[*range(10,10,1)]

generate playlist ID value range to create correct submission format.  We don't want to use a naive range of ids like the following

    pid = pd.DataFrame([*range(1000000,1010000)]) 

We want to use the original pids from the challenge set.
The pid is not arbitrary and should match the order of the pid in the challenge_playlists.  
The rows of the solution are in the order of the original challenge set to should apply directly.

In [ ]:
pid = challenge_playlists["pid"]

In [ ]:
pid=pid.to_frame()

In [ ]:
pid.shape

In [ ]:
pid

In [ ]:
solution.insert(0, "pid", pid) #[*range(2000000,2010000)])

In [ ]:
solution#[998:1005]

## Write solution to output file

Include only the data not any index or headers.

In [ ]:
solution_csv = solution.to_csv(index=False,header=False)

In [ ]:
text_file = open(challenge_solution, "w")
n = text_file.write(challenge_header+solution_csv)
text_file.close()